# Calculating Lead-Based Returns with Rule-Based Classification using Gezinomi's Dataset

# Task 1: 
Answer the following questions.

Question 1: Read gezinomi.xlsx and show general information about the dataset.

In [1]:
# Libraries
import pandas as pd

# Settings
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [2]:
df = pd.read_excel('gezinomi.xlsx')

In [3]:
def check_df(dataframe, head=7, tail=7):
    '''
    Prints the general information about the given dataframe e.g. shape, head,
    tail, info, descriptive statistics, etc.

    Parameters
    ----------
    dataframe : DataFrame
        The dataframe that we want to have general information about.
    head: int
        Prints the first n rows of the dataframe.
    tail: int
        Prints the last n rows of the dataframe.
    '''
    print('####### Shape #######')
    print(dataframe.shape)
    print('####### Info #######')
    print(dataframe.info())
    print('####### Head #######')
    print(dataframe.head(head))
    print('####### Tail #######')
    print(dataframe.tail(tail))
    print('####### Descriptive Statistics #######')
    print(dataframe.describe([0.05, 0.25, 0.50, 0.75 ,0.95, 0.99]).T)
    print('####### NA #######')
    print(dataframe.isnull().sum())
    print('####### Number of Unique Values #######')
    print(dataframe.nunique())


check_df(df)

####### Shape #######
(59164, 9)
####### Info #######
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59164 entries, 0 to 59163
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SaleId              59164 non-null  int64         
 1   SaleDate            59164 non-null  datetime64[ns]
 2   CheckInDate         59164 non-null  datetime64[ns]
 3   Price               59151 non-null  float64       
 4   ConceptName         59164 non-null  object        
 5   SaleCityName        59164 non-null  object        
 6   CInDay              59164 non-null  object        
 7   SaleCheckInDayDiff  59164 non-null  int64         
 8   Seasons             59164 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 4.1+ MB
None
####### Head #######
   SaleId   SaleDate CheckInDate   Price     ConceptName SaleCityName    CInDay  SaleCheckInDayDiff Seasons
0  4151

Question 2: How many unique 'cities' are there? What are their frequencies?

In [4]:
# Unique values
df['SaleCityName'].unique()

array(['Antalya', 'İzmir', 'Diğer', 'Aydın', 'Muğla', 'Girne'],
      dtype=object)

In [5]:
# Frequencies
df['SaleCityName'].value_counts()

Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: SaleCityName, dtype: int64

Question 3: How many unique 'concepts' are there?

In [6]:
df['ConceptName'].unique()

array(['Herşey Dahil', 'Yarım Pansiyon', 'Oda + Kahvaltı'], dtype=object)

Question 4: How many sales were made from which 'Concept'?

In [7]:
df.groupby('ConceptName').agg({'Price': 'count'})

,Price
ConceptName,
Herşey Dahil,53180
Oda + Kahvaltı,2414
Yarım Pansiyon,3557


Question 5: How many sales were made from which 'city'?

In [8]:
df.groupby('SaleCityName').agg({'Price': 'sum'})

,Price
SaleCityName,
Antalya,2041911.096
Aydın,573296.009
Diğer,154572.290
Girne,27065.028
Muğla,665842.213
İzmir,165934.834


Question 6: How much was earned in total from sales by 'concept'?

In [9]:
df.groupby('ConceptName').agg({'Price': 'sum'})

,Price
ConceptName,
Herşey Dahil,3332910.768
Oda + Kahvaltı,121308.353
Yarım Pansiyon,174402.350


Question 7: What are the 'Price' averages by 'cities'?

In [10]:
df.groupby('SaleCityName').agg({'Price': 'mean'})

,Price
SaleCityName,
Antalya,64.521
Aydın,53.856
Diğer,47.707
Girne,59.484
Muğla,62.462
İzmir,66.268


Question 8: What are the 'Price' averages according by 'concepts'?

In [11]:
df.groupby('ConceptName').agg({'Price': 'mean'})

,Price
ConceptName,
Herşey Dahil,62.672
Oda + Kahvaltı,50.252
Yarım Pansiyon,49.031


Question 9: What are the 'Price' averages in the 'City-Concept' breakdown?

In [12]:
df.groupby(['SaleCityName', 'ConceptName']).agg({'Price': 'mean'})

Price
SaleCityName ConceptName          
Antalya      Herşey Dahil   64.519
             Oda + Kahvaltı 63.505
             Yarım Pansiyon 67.191
Aydın        Herşey Dahil   53.995
             Oda + Kahvaltı 34.459
             Yarım Pansiyon 30.017
Diğer        Herşey Dahil   84.771
             Oda + Kahvaltı 37.599
             Yarım Pansiyon 42.113
Girne        Herşey Dahil   97.681
             Oda + Kahvaltı 39.776
             Yarım Pansiyon 53.248
Muğla        Herşey Dahil   63.020
             Oda + Kahvaltı 59.038
             Yarım Pansiyon 45.121
İzmir        Herşey Dahil   74.702
             Oda + Kahvaltı 41.320
             Yarım Pansiyon 59.610

# Task 2: 
Convert the variable 'SaleCheckInDayDiff' to a new categorical variable named 'EB_Score'.
- The 'SaleCheckInDayDiff' variable indicates how long before the CheckIn date ('CInDay') the customer completed their purchase.
- Construct the intervals convincingly. 
  For Example: You can use the ranges '0_7', '7_30', '30_90', '90_max'.
- You can use the names 'Last Minuters', 'Potential Planners', 'Planners', 'Early Bookers' for these intervals.

In [13]:
df['EB_Score'] = pd.cut(df['SaleCheckInDayDiff'], bins=[-1, 7, 30, 90, df['SaleCheckInDayDiff'].max()], labels=['Last Minuters', 'Potential Planners', 'Planners', 'Early Bookers'])
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons,EB_Score
0,415122,2022-12-03,2022-12-03,79.304,Herşey Dahil,Antalya,Saturday,0,Low,Last Minuters
1,415103,2022-12-03,2022-12-03,45.971,Yarım Pansiyon,Antalya,Saturday,0,Low,Last Minuters
2,404034,2022-09-12,2022-09-13,77.839,Herşey Dahil,Antalya,Tuesday,1,High,Last Minuters
3,415094,2022-12-03,2022-12-10,222.711,Yarım Pansiyon,İzmir,Saturday,7,Low,Last Minuters
4,414951,2022-12-01,2022-12-03,140.476,Yarım Pansiyon,İzmir,Saturday,2,Low,Last Minuters


# Task 3: 
Look at the average wages and frequencies in the breakdown of 'City', 'Concept', [EB_Score, Season, CInday].
- 'Price' averages and frequencies in the breakdown of 'City-Concept-EB_Score'.
- 'Price' averages and frequencies in the breakdown of 'City-Concept-Season'. 
- 'Price' averages and frequencies in the breakdown of 'City-Concept-CInday'.

In [14]:
# 'Price' averages and frequencies in the breakdown of 'City-Concept-EB_Score'.
df.groupby(['SaleCityName', 'ConceptName', 'EB_Score']).agg({'Price': ['mean', 'count']})

Price       
                                                 mean  count
SaleCityName ConceptName    EB_Score                        
Antalya      Herşey Dahil   Last Minuters      62.753  14148
                            Potential Planners 64.904   8874
                            Planners           67.880   4490
                            Early Bookers      66.495   3281
             Oda + Kahvaltı Last Minuters      65.354    503
...                                               ...    ...
İzmir        Oda + Kahvaltı Early Bookers      66.928      3
             Yarım Pansiyon Last Minuters      57.788    636
                            Potential Planners 64.804    198
                            Planners           60.508     47
                            Early Bookers      65.906     14

[72 rows x 2 columns]

In [15]:
# 'Price' averages and frequencies in the breakdown of 'City-Concept-Season'
df.groupby(['SaleCityName', 'ConceptName', 'Seasons']).agg({'Price': ['mean', 'count']})

Price       
                                       mean  count
SaleCityName ConceptName    Seasons               
Antalya      Herşey Dahil   High     64.920  27126
                            Low      61.552   3667
             Oda + Kahvaltı High     66.268    303
                            Low      60.667    295
             Yarım Pansiyon High     73.259    118
                            Low      62.002    138
Aydın        Herşey Dahil   High     54.947  10103
                            Low      33.681    473
             Oda + Kahvaltı High     30.388     27
                            Low      44.452     11
             Yarım Pansiyon High     32.626     20
                            Low      25.272     11
Diğer        Herşey Dahil   High     83.787    364
                            Low      87.311    141
             Oda + Kahvaltı High     34.840    349
                            Low      39.960    408
             Yarım Pansiyon High     39.732    516
                            Low      42.954   1462
Girne        Herşey Dahil   High    103.935     55
                            Low      90.936     51
             Oda + Kahvaltı High     39.480     66
                            Low      40.043     73
             Yarım Pansiyon High     58.384    100
                            Low      48.579    110
Muğla        Herşey Dahil   High     63.384   9582
                            Low      48.304    237
             Oda + Kahvaltı High     61.265    592
                            Low      37.774     62
             Yarım Pansiyon High     45.824    177
                            Low      32.682     10
İzmir        Herşey Dahil   High     74.748   1234
                            Low      74.308    147
             Oda + Kahvaltı High     54.142     86
                            Low      33.555    142
             Yarım Pansiyon High     87.657    118
                            Low      55.351    777

In [16]:
# 'Price' averages and frequencies in the breakdown of 'City-Concept-CInday'.
df.groupby(['SaleCityName', 'ConceptName', 'CInDay']).agg({'Price': ['mean', 'count']})

Price      
                                        mean count
SaleCityName ConceptName    CInDay                
Antalya      Herşey Dahil   Friday    62.659  4136
                            Monday    63.259  6831
                            Saturday  64.416  4741
                            Sunday    65.849  3818
                            Thursday  62.893  3898
...                                      ...   ...
İzmir        Yarım Pansiyon Saturday  52.505   315
                            Sunday    98.935    68
                            Thursday  52.485    91
                            Tuesday   56.141    73
                            Wednesday 54.064    86

[126 rows x 2 columns]

# Task 4: 
Sort the output of the 'City-Concept-Season' breakdown according to 'Price'.
- To better see the output from the previous question, apply the sort_values() ​​method to 'Price' in descending order.
- Save the output as agg_df.

In [17]:
agg_df = df.groupby(['SaleCityName', 'ConceptName', 'Seasons']).agg({'Price': 'mean'}).sort_values('Price', ascending=False)
agg_df.head()

Price
SaleCityName ConceptName    Seasons        
Girne        Herşey Dahil   High    103.935
                            Low      90.936
İzmir        Yarım Pansiyon High     87.657
Diğer        Herşey Dahil   Low      87.311
                            High     83.787

# Task 5: 
Convert the names in the index to variable names.
- All variables except 'Price' in the output of the third question are index names.
Convert these names to variable names.

In [18]:
agg_df = agg_df.reset_index()
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.935
1,Girne,Herşey Dahil,Low,90.936
2,İzmir,Yarım Pansiyon,High,87.657
3,Diğer,Herşey Dahil,Low,87.311
4,Diğer,Herşey Dahil,High,83.787


# Task 6: 
Define new level-based customers (persona)
- Define new level-based customers (persona) and add them as variables to the dataset.
- Name of the new variable to be added: 'sales_level_based'
- You need to create the 'sales_level_based' variable by combining the observations from the output from the previous question.

In [19]:
agg_df['sales_level_based'] = agg_df[['SaleCityName', 'ConceptName', 'Seasons']].agg(lambda x: '_'.join(x).upper(), axis=1)
agg_df = agg_df[['sales_level_based', 'Price']]
agg_df.head()

,sales_level_based,Price
0,GIRNE_HERŞEY DAHIL_HIGH,103.935
1,GIRNE_HERŞEY DAHIL_LOW,90.936
2,İZMIR_YARIM PANSIYON_HIGH,87.657
3,DIĞER_HERŞEY DAHIL_LOW,87.311
4,DIĞER_HERŞEY DAHIL_HIGH,83.787


In [20]:
# Check duplicate values
agg_df['sales_level_based'].value_counts().head(10)

GIRNE_HERŞEY DAHIL_HIGH      1
GIRNE_HERŞEY DAHIL_LOW       1
MUĞLA_HERŞEY DAHIL_LOW       1
MUĞLA_YARIM PANSIYON_HIGH    1
AYDIN_ODA + KAHVALTI_LOW     1
DIĞER_YARIM PANSIYON_LOW     1
GIRNE_ODA + KAHVALTI_LOW     1
DIĞER_ODA + KAHVALTI_LOW     1
DIĞER_YARIM PANSIYON_HIGH    1
GIRNE_ODA + KAHVALTI_HIGH    1
Name: sales_level_based, dtype: int64

# Task 7: 
Segment new customers (personas).
- Divide new personas into 4 segments according to 'Price'.
- Add the segments to agg_df as a variable with the 'SEGMENT' naming.
- Describe segments (groupby() segments and get 'Price' mean, max, sum).

In [21]:
# Segments
agg_df['SEGMENT'] = pd.qcut(agg_df['Price'], 4, labels=['D', 'C', 'B', 'A'])
agg_df.head()

,sales_level_based,Price,SEGMENT
0,GIRNE_HERŞEY DAHIL_HIGH,103.935,A
1,GIRNE_HERŞEY DAHIL_LOW,90.936,A
2,İZMIR_YARIM PANSIYON_HIGH,87.657,A
3,DIĞER_HERŞEY DAHIL_LOW,87.311,A
4,DIĞER_HERŞEY DAHIL_HIGH,83.787,A


In [22]:
agg_list = ['count', 'min', 'max', 'mean', 'median', 'std']
agg_df.groupby('SEGMENT').agg({'Price': agg_list})

Price                                    
        count    min     max   mean median    std
SEGMENT                                          
D           9 25.272  39.480 33.366 33.555  4.101
C           9 39.732  54.142 44.888 44.452  4.874
B           9 54.947  64.920 60.274 61.265  3.413
A           9 66.268 103.935 82.468 83.787 11.501

# Task 8: 
Classify new customers and estimate how much revenue they can generate.
- How much income is expected to be earned on average by a person who wants to have an all-inclusive holiday in Antalya during the high season?
- In which segment will a holidaymaker go to a half-board hotel in Girne in low season?

In [23]:
# Antalya
new_user = 'ANTALYA_HERŞEY DAHIL_HIGH'
agg_df[agg_df['sales_level_based'] == new_user]

,sales_level_based,Price,SEGMENT
9,ANTALYA_HERŞEY DAHIL_HIGH,64.920,B


In [24]:
# Girne
new_user = 'GIRNE_YARIM PANSIYON_LOW'
agg_df[agg_df['sales_level_based'] == new_user]

,sales_level_based,Price,SEGMENT
19,GIRNE_YARIM PANSIYON_LOW,48.579,C
